## Introduction to Creating RAGs (Retrieval-Augmented Generators) with OpenAI

## 1. - Langchan LLM Chain Tutorial

## 1.1 - Build a basic agent

In [1]:
from langchain.agents import create_agent
import os
import getpass

os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY")

/home/tulio/Documents/Repositories/TDSE-04-RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    tools=[get_weather],
    system_prompt="You are a helpful assistant",

)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='ef80654e-7182-4154-924f-d08ed8419cef'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "sf"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c6234-94e4-7a11-b171-3f8e2062113c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': '376d42b0-f3e9-4d08-8f2c-f001908d432d', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 51, 'output_tokens': 15, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in sf!", name='get_weather', id='ac0ce749-ebdb-4ed6-a1fc-99e0bbee280a', tool_call_id='376d42b0-f3e9-4d08-8f2c-f001908d432d'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_n

## 1.2 - Build a real-world agent

In [4]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. 
If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [5]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city. """
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema. """
    user_id: str


@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [6]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite",
    temperature=0.5,
    timeout=10,
    max_tokens=10000
)

In [7]:
@dataclass 
class ResponseFormat:
    """Response schema for the agent."""
    punny_response: str
    weather_conditions: str | None = None

In [8]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [9]:
from langchain.agents.structured_output import ToolStrategy 

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])

response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you"}]},
    config = config,
    context = Context(user_id="1")
)

print(response['structured_response'])

ResponseFormat(punny_response="Looks like you're in for some fun in the sun! It's always sunny in Florida!", weather_conditions=None)
ResponseFormat(punny_response="You're welcome! If you need any more weather puns, just ask!", weather_conditions=None)
